In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import csv

In [ ]:
df_hospital=pd.read_csv("processed_data/dublinbikes_james_hospital.csv")
df_portobello=pd.read_csv("processed_data/dublinbikes_portobello.csv")